In [4]:
import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict
from statistics import harmonic_mean

import timeit
start = timeit.default_timer()

randomlist = []
demand = {}
random.seed(1) # Use the same random seed
for i in range(0,20):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(20):
    demand["D"+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in demand.items():
    sum = sum + v

supply = {}
for i in range(sum):
    supply[i] = 1.0

np.random.seed(1) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,20, size=(sum,20))).astype(float)

random.seed(1) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(1) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(1) # Use the same random seed
doc = {}
for i in range(0,20):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom
df_tdkom.columns = [("D"+str(i)) for i in range(1,21)]
dk = df_tdkom.transpose()
costs = dk.to_dict()
#costs_c = df_tdkom.to_dict()
cols = sorted(demand.keys())

#print(supply['S1'])
# #break
costs1=copy.deepcopy(costs)
# costs2=copy.deepcopy(costs)
# costs3=copy.deepcopy(costs)

# for i in supply:
#     mi=min(costs[i].values())
#     #print(costs[i])
#     # print(mi)
#     for j in costs2[i]:
#         costs2[i][j]-=mi
# # print(costs2)
# for i in demand :
#     mi=10000
#     for j in supply:
#         if costs[j][i]<mi :
#             mi=costs[j][i]
#     for j in supply:
#         costs3[j][i]=costs3[j][i]-mi
# # print(costs3)

# for i in demand:
#     for j in supply:
#         costs[j][i]= costs2[j][i]+costs3[j][i]

res = dict((k, defaultdict(int)) for k in costs)
ls=len(supply)

while supply and demand:
    sd={}
    # print(demand,supply)
    for x in demand:
        # print(x)
        l=[]
        # print(len(supply))
        for y in supply:
            l.append(costs[y][x])
        # print(l)
        if len(supply)>1:

            sd[x]=harmonic_mean(l)
        else:
            sd[x]=(list(supply.values()))[0]

    for y in supply :
        l=[]
        for x in demand:
            l.append(costs[y][x])
        if len(demand)>1:

            sd[y]=harmonic_mean(l)
        else:
            sd[y]=(list(demand.values()))[0]
        # sd[y]=stdev(l)
    # print(sd)
    ma=max(sd.values())
    l=[]
    for ke in sd:
        if sd[ke]==ma:
            l.append(ke)
    ind=l[0]
    # # print(l)
    # mi=1000000
    # for ke in l:
    # 	if demand.get(ke)!=None:
    # 		for y in supply:
    # 			mi=min(mi,costs[y][ke])
    # 	else:
    # 		for x in demand:
    # 			mi=min(mi,costs[ke][x])
    # nl=[]
    # # print(mi)
    # flagg=0
    # if len(l)==1:
    # 	nl=l
    # 	flagg=1

    # for ke in l :
    # 	if flagg==1:
    # 		break
    # 	if demand.get(ke)!=None:
    # 		mi2=19000
    # 		for y in supply:
    # 			mi2=min(mi2,costs[y][ke])
    # 		if mi2==mi:
    # 			nl.append(ke)
    # 	else:
    # 		mi2=190000
    # 		for y in demand:
    # 			mi2=min(mi2,costs[ke][y])
    # 		if mi2==mi:
    # 			nl.append(ke)
    # # print(nl)
    # ind="D"
    # mi3=100000
    # flagg=0
    # if len(nl)==1:
    # 	ind=nl[0]
    # 	flagg=1

    # for ke in nl :
    # 	if flagg==1:
    # 		break
    # 	if demand.get(ke)!=None:

    # 		mi2=10000
    # 		for y in supply:
    # 			if costs[y][ke]<mi2:
    # 				mi2=costs[y][ke]
    # 		if mi2<mi3:
    # 			mi3=mi2
    # 			ind=ke
    # 	else:
    # 		mi2=10000
    # 		for y in demand:
    # 			if costs[ke][y]<mi2:
    # 				mi2=costs[ke][y]
    # 		if mi2<mi3:
    # 			mi3=mi2
    # 			ind=ke
    # print(ind)
    if demand.get(ind)!=None:
        mi=10000
        mind="S"
        for y in supply:
            if costs[y][ind]<mi:
                mi=costs[y][ind]
                mind=y
        v=min(supply[mind],demand[ind])
        supply[mind]-=v
        demand[ind]-=v
        res[mind][ind]+=v
        if supply[mind]==0:
            del supply[mind]
        if demand[ind]==0:
            del demand[ind]
    else:
        mi=10000
        mind="S"
        # print(ind)
        for y in demand:
            # print(ind,y)
            if costs[ind][y]<mi:
                mi=costs[ind][y]
                mind=y
        v=min(supply[ind],demand[mind])
        supply[ind]-=v
        demand[mind]-=v
        res[ind][mind]+=v
        if supply[ind]==0:
            del supply[ind]
        if demand[mind]==0:
            del demand[mind]

    # break


# print(demand,supply)

cost = 0
for g in sorted(costs1):
    # print (g, "\t",)
    for n in cols:
        y = res[g][n]
        if y != 0:
            pass
            # print (y,)
        cost += y * costs1[g][n]
        # print ("\t",)
    # print()
# print ("\n\nTotal Cost = ", cost)

In [5]:
df_tdkom

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D20
0,245.590411,410.654755,99.177855,202.690808,148.328800,122.813228,172.992106,247.817795,234.923124,292.256049,281.587404,374.943082,186.288507,435.714300,94.895220,383.087460,250.430971,346.581516,165.910961,146.994306
1,539.251046,672.718976,286.171482,481.977882,601.474036,609.814179,147.209600,127.212542,168.830243,579.143451,164.185709,332.498289,662.404977,376.396558,491.233659,285.280968,477.263214,599.579803,120.722027,504.516460
2,464.849607,403.671974,211.325887,390.593656,126.614605,262.976194,458.420420,217.871641,180.779516,114.042822,109.894842,355.047324,183.566724,167.249303,280.343196,117.859920,302.719350,166.607348,342.462674,343.355666
3,117.401012,300.865150,429.614476,271.305372,110.616792,338.741734,409.493015,345.742208,514.754823,341.705628,532.317471,148.983804,166.313749,445.108651,272.996235,176.642219,513.398112,274.323857,463.246269,407.938153
4,447.035753,371.197968,421.667038,223.587104,201.694136,468.194217,277.420074,514.765793,350.596878,329.394917,152.072849,490.376086,292.324718,309.486459,258.541027,198.871166,460.375528,354.622081,107.314639,327.552848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,454.972432,260.405521,663.848771,432.223273,669.185522,443.459237,424.543577,216.675547,130.809290,234.913263,113.722840,552.792332,211.445771,381.859615,661.485524,376.261235,197.242009,206.026431,203.850455,245.286578
2029,377.730076,323.164396,353.199525,345.870285,457.869603,149.042333,308.683739,482.748522,108.687056,355.311880,287.995624,88.147448,218.579754,429.499734,117.638279,225.452874,210.142512,337.239231,422.061710,75.082801
2030,387.933501,261.277135,460.832591,250.513121,327.846885,347.767889,154.715610,224.011209,252.410506,257.517075,355.179351,145.366938,214.728010,63.941734,179.668635,471.866825,129.067548,122.268898,124.353683,240.280743
2031,268.663235,121.862345,324.262275,430.109930,488.937627,180.296280,405.047590,434.277218,401.726874,158.402932,206.845186,147.267560,424.649308,123.978872,276.596119,166.100640,358.652372,501.054962,209.945939,403.265537


In [6]:
cost

253617.8079350977